In [1]:
import pandas as pd

def clean_csv_data(input_csv, output_csv="cleaned_data.csv"):
    '''
    Args:
        input_csv (str): Path to the uncleaned CSV file.
        output_csv (str): Path to save the cleaned data (default: "cleaned_data.csv").
    Returns:
        pd.DataFrame: A cleaned DataFrame with pivoted categories.
    '''
    df = pd.read_csv(input_csv)
    
    # Ensure required columns exist
    required_columns = ["book_name", "author_name", "category_name", "category_description", "categories_data"]
    for col in required_columns:
        if col not in df.columns:
            raise KeyError(f"Missing required column: {col}")
    
    # Fill missing values and convert to string
    df["category_description"] = df["category_description"].fillna("").astype(str)
    df["categories_data"] = df["categories_data"].fillna("").astype(str)
    
    # Clean and split categories
    df["category_description"] = df["category_description"].str.replace("\n", "", regex=True).str.strip().str.split(", ")
    df["categories_data"] = df["categories_data"].str.replace("\n", "", regex=True).str.replace("%", "").str.strip().str.split(", ")
    
    # Ensure lists have matching lengths
    def adjust_lengths(row):
        names, values = row["category_description"], row["categories_data"]
        if isinstance(names, list) and isinstance(values, list):
            min_len = min(len(names), len(values))
            return names[:min_len], values[:min_len]
        return names, values
    
    df[["category_description", "categories_data"]] = df.apply(adjust_lengths, axis=1, result_type="expand")
    
    # Explode the DataFrame to make each category a row
    df_rows = df.explode(["category_description", "categories_data"])
    
    # Convert category data to numeric values
    df_rows["categories_data"] = pd.to_numeric(df_rows["categories_data"], errors="coerce")
    
    # Aggregate duplicates by taking the mean
    df_rows = df_rows.groupby(["book_name", "author_name", "category_description"], as_index=False).mean(numeric_only=True)
    
    # Pivot the categories into columns
    df_pivot = df_rows.pivot(index=['book_name', 'author_name'], columns="category_description", values="categories_data")
    
    # Reset index to bring "book_name" and "author_name" back as columns
    df_pivot.reset_index(inplace=True)
    
    # Fill NaN values with 0
    df_pivot.fillna(0, inplace=True)
    
    # Save to CSV
    df_pivot.to_csv(output_csv, index=False)
    
    return df_pivot

clean_csv_data("output_new.csv")

category_description,book_name,author_name,Accurate,Accurate,Action,Actionable,Adult themes,Adult themes,Advanced,Angry,...,Meandering,Medium,Medium paced,Mystery,Neutral,Recommend,Romantic,Slow paced,and figures,diagrams
0,A Christmas carol,\nCharles Dickens\n\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Adventures of Huckleberry Finn,\nMark Twain\n\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Adventures of Sherlock Holmes,\nArthur Conan Doyle\n\n,0.0,33.0,12.0,33.0,0.0,14.0,0.0,12.0,...,0.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,50.0,16.0
3,Animal Farm,\nGeorge Orwell\n\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Charlie and the Chocolate Factory,\nRoald Dahl\n\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,Charlie and the Great Glass Elevator,\nRoald Dahl\n\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,Cinquante nuances de Grey,\nE. L. James\n\n,0.0,0.0,0.0,0.0,100.0,0.0,100.0,0.0,...,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,Daddy-Long-Legs,\nJean Webster\n\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,El Principito / The Little Prince,\nAntoine de Saint-Exupéry\n\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,Emma,\nJane Austen\n\n,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
